<a href="https://colab.research.google.com/github/Firojpaudel/Reasoning_Model_/blob/main/Creating_reasoning_model_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Before Diving In:

**What is GRPO?**
- Stands for Group Relative Policy Optimization. Its is a reinforcement learning technique that helps models develop reasoning capabilities autonomously.
- First introduced the concept on DeepSeek paper.

---

#### **Why Unsloth?**:
- Well, thanks to Unsloth, training a larger params model ( $15B$ or so) on customer-grade GPUs like that of Colab and Kaggle is possible
  - Reason: Due to some major optimizations done on the **transformers** library.
---
> Major Props to Daniel and team 🙌!

In [1]:
!rm -rf /content/sample_data

In [2]:
## Installation
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

#### The Main Process:


In [4]:
!pip install langid -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-0528-Qwen3-8B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6 # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-11 17:32:42 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-11 17:32:42 [__init__.py:239] Automatically detected platform cuda.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit with actual GPU utilization = 59.43%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.24 GB. Also swap space = 0 GB.


...

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/5.26k [00:00<?, ?B/s]

Unsloth 2025.6.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


#### Optimizing for GRPO Chat Template


In [6]:
reasoning_start = None
reasoning_end = None
user_token = None
assistant_token = None

for token in tokenizer.get_added_vocab().keys():
    if "think" in token and "/" in token:
        reasoning_end = token
    elif "think" in token:
        reasoning_start = token
    elif "user" in token:
        user_token = token
    elif "assistant" in token:
        assistant_token = token

system_prompt = \
f"""You are SAGE nano. A model from SAGEA.
You are given a problem.
Think about the problem and provide your working out.
You must think in English.
You are capable of providing answers in Nepali as well.
If users prompt in Nepali, provide answer them in Nepali as well.
"""
system_prompt

'You are SAGE nano. A model from SAGEA.\nYou are given a problem.\nThink about the problem and provide your working out.\nYou must think in English.\nYou are capable of providing answers in Nepali as well.\nIf users prompt in Nepali, provide answer them in Nepali as well.\n'

In [7]:
print(tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"<think>I think it's 2.2</think>2"},
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"<think>I think it's 2.2</think>2"},
], tokenize = False, add_generation_prompt = True))

<｜begin▁of▁sentence｜><｜User｜>What is 1+1?<｜Assistant｜>2<｜end▁of▁sentence｜><｜User｜>What is 1+1?<｜Assistant｜>2<｜end▁of▁sentence｜><｜Assistant｜>


#### Data prep:


In [50]:
from datasets import load_dataset, concatenate_datasets, Dataset

# Function to format datasets for GRPO
def format_example(example, dataset_name):
    prompt = ""
    reasoning = "Reasoning: Default reasoning if none provided."
    answer = ""

    if dataset_name == "open-r1/DAPO-Math-17k-Processed":
        prompt = example.get("question", "") or example.get("prompt", "")
        reasoning = example.get("reasoning", "Reasoning: Break down the math problem step by step.")
        answer = str(example.get("answer", "") or example.get("solution", ""))
    elif dataset_name == "cais/mmlu":
        prompt = example.get("question", "")
        reasoning = "Reasoning: Analyze the question and deduce the correct solution."
        answer = str(example.get("answer", ""))
    elif dataset_name == "glue":
        premise = example.get("premise", "").strip()
        hypothesis = example.get("hypothesis", "").strip()
        if not premise or not hypothesis:
            return None
        prompt = f"Premise: {premise} Hypothesis: {hypothesis}"
        reasoning = "Reasoning: Determine if the hypothesis entails, contradicts, or is neutral."
        answer = str(example.get("label", ""))
    elif dataset_name == "xnli":
        premise = example.get("premise", {}).get("en", "").strip()
        hypothesis_dict = example.get("hypothesis", {})
        if not premise or not isinstance(hypothesis_dict, dict) or "language" not in hypothesis_dict or "translation" not in hypothesis_dict:
            print(f"Skipping invalid xnli example: premise={premise}, hypothesis={hypothesis_dict}")
            return None
        # Extract English hypothesis
        try:
            en_idx = hypothesis_dict["language"].index("en")
            hypothesis = hypothesis_dict["translation"][en_idx].strip()
        except (ValueError, IndexError):
            print(f"Skipping xnli example, no English hypothesis: {hypothesis_dict}")
            return None
        if not hypothesis:
            print(f"Skipping xnli example, empty hypothesis: {hypothesis_dict}")
            return None
        prompt = f"Premise: {premise} Hypothesis: {hypothesis}"
        reasoning = "Reasoning: Evaluate for entailment, contradiction, or neutrality."
        answer = str(example.get("label", ""))
    elif dataset_name == "truthful_qa":
        prompt = example.get("question", "").strip()
        reasoning = "Reasoning: Critically assess the question for a truthful answer."
        answer = str(example.get("best_answer", ""))
    elif dataset_name == "lukaemon/bbh":
        prompt = example.get("input", "").strip()
        reasoning = "Reasoning: Tackle this complex task step by step."
        answer = str(example.get("target", ""))
    elif dataset_name == "opus100":
        prompt = example.get("translation", {}).get("en", "").strip()
        reasoning = "Reasoning: Translate and respond naturally in Nepali."
        answer = str(example.get("translation", {}).get("ne", ""))
    else:
        prompt = example.get("prompt", "").strip()
        reasoning = example.get("reasoning", reasoning)
        answer = str(example.get("answer", ""))

    if not prompt or not answer.strip():
        return None

    chosen = f"<reasoning>{reasoning}</reasoning><answer>{answer}</answer>"
    if not chosen.strip() or "<answer></answer>" in chosen:
        return None

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": "<reasoning>Incomplete or incorrect reasoning.</reasoning><answer>Wrong or irrelevant response.</answer>"
    }

In [51]:
# Load and format datasets
def filter_valid_examples(dataset, dataset_name):
    print(f"Processing {dataset_name}, first example: {dataset[0] if len(dataset) > 0 else 'Empty dataset'}")

    formatted = dataset.map(
        lambda x: format_example(x, dataset_name),
        remove_columns=dataset.column_names
    )
    filtered = formatted.filter(
        lambda x: (
            x is not None and
            isinstance(x, dict) and
            "prompt" in x and x["prompt"] and x["prompt"].strip() and
            "chosen" in x and x["chosen"] and "<answer></answer>" not in x["chosen"] and
            "rejected" in x and x["rejected"]
        )
    )

    if filtered.num_rows == 0:
        print(f"Warning: No valid examples for {dataset_name} after filtering")
        return None

    if set(filtered.column_names) != {"prompt", "chosen", "rejected"}:
        print(f"Warning: Unexpected columns in {dataset_name}: {filtered.column_names}")
        return None

    print(f"Filtered {dataset_name}: {filtered.num_rows} examples")
    return filtered

In [52]:
# Load datasets
datasets_to_concat = []
dapo_math = filter_valid_examples(load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split="train"), "open-r1/DAPO-Math-17k-Processed")
if dapo_math:
    datasets_to_concat.append(dapo_math)

mmlu = filter_valid_examples(load_dataset("cais/mmlu", "all", split="auxiliary_train"), "cais/mmlu")
if mmlu:
    datasets_to_concat.append(mmlu)

multinli = filter_valid_examples(load_dataset("glue", "mnli", split="train[:10000]"), "glue")
if multinli:
    datasets_to_concat.append(multinli)

xnli = filter_valid_examples(load_dataset("xnli", "all_languages", split="train[:10000]"), "xnli")
if xnli:
    datasets_to_concat.append(xnli)

truthfulqa = filter_valid_examples(load_dataset("truthful_qa", "generation", split="validation"), "truthful_qa")
if truthfulqa:
    datasets_to_concat.append(truthfulqa)

bbh_navigate = filter_valid_examples(load_dataset("lukaemon/bbh", "navigate", split="test"), "lukaemon/bbh")
if bbh_navigate:
    datasets_to_concat.append(bbh_navigate)

bbh_boolean = filter_valid_examples(load_dataset("lukaemon/bbh", "boolean_expressions", split="test"), "lukaemon/bbh")
if bbh_boolean:
    datasets_to_concat.append(bbh_boolean)

opus = filter_valid_examples(
    load_dataset("opus100", "en-ne", split="train[:5000]").filter(lambda x: x["translation"]["en"] and x["translation"]["ne"]),
    "opus100"
)
if opus:
    datasets_to_concat.append(opus)

Processing open-r1/DAPO-Math-17k-Processed, first example: {'prompt': 'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.', 'solution': '34', 'data_source': 'math_dapo', 'source_prompt': [{'content': 'Solve the following math problem step by step. The last line of your response should be of the form Answer: $Answer (without quotes) where $Answer is the answer to the problem.\n\nIn triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can b

Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14116 [00:00<?, ? examples/s]

Filtered open-r1/DAPO-Math-17k-Processed: 14116 examples
Processing cais/mmlu, first example: {'question': "Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were", 'subject': '', 'choices': ['Adams only.', 'Brooks only.', 'Case only.', 'Adams and Brooks'], 'answer': 1}


Map:   0%|          | 0/99842 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/250 [00:00<?, ? examples/s]

Filtered lukaemon/bbh: 250 examples
Processing opus100, first example: {'translation': {'en': '_Inv', 'ne': 'Inv'}}


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filtered opus100: 5000 examples


In [53]:
# Verify schema consistency and log sizes
for ds in datasets_to_concat:
    print(f"Dataset {ds}: Features: {ds.features}, Num rows: {ds.num_rows}")

Dataset Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 14116
}): Features: {'prompt': Value(dtype='string', id=None), 'chosen': Value(dtype='string', id=None), 'rejected': Value(dtype='string', id=None)}, Num rows: 14116
Dataset Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 99842
}): Features: {'prompt': Value(dtype='string', id=None), 'chosen': Value(dtype='string', id=None), 'rejected': Value(dtype='string', id=None)}, Num rows: 99842
Dataset Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 10000
}): Features: {'prompt': Value(dtype='string', id=None), 'chosen': Value(dtype='string', id=None), 'rejected': Value(dtype='string', id=None)}, Num rows: 10000
Dataset Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 10000
}): Features: {'prompt': Value(dtype='string', id=None), 'chosen': Value(dtype='string', id=None), 'rejected': Value(dtype='string', id=None)}, Num rows: 10000
Dataset Dataset(

In [54]:
# Combine and shuffle
if not datasets_to_concat:
    raise ValueError("No valid datasets to concatenate")
combined_dataset = concatenate_datasets(datasets_to_concat)
combined_dataset = combined_dataset.select_columns(["prompt", "chosen", "rejected"])
combined_dataset = combined_dataset.shuffle(seed=3407)
combined_dataset = combined_dataset.filter(
    lambda x: (
        x["prompt"] and x["prompt"].strip() and
        x["chosen"] and "<answer></answer>" not in x["chosen"] and
        x["rejected"]
    )
)

Filter:   0%|          | 0/140275 [00:00<?, ? examples/s]

In [55]:
#Inspecting ...
combined_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 140275
})

In [56]:
combined_dataset[5000]

{'prompt': "An application wants access to the keyring '%s', but it is locked",
 'chosen': "<reasoning>Reasoning: Translate and respond naturally in Nepali.</reasoning><answer>एउटा अज्ञात अनुप्रयोगले '%s' किरिङमा पहुँच प्राप्त गर्न चाहेको छ, तर यसलाई ताल्चा लगाइएको छ</answer>",
 'rejected': '<reasoning>Incomplete or incorrect reasoning.</reasoning><answer>Wrong or irrelevant response.</answer>'}

#### Creating a regex format to match the reasoning selections and answers:


In [26]:
import re

# Add optional EOS token matching
solution_end_regex = rf"{reasoning_end}(.*)"

match_format = re.compile(solution_end_regex, re.DOTALL)
match_format

re.compile(r'</think>(.*)', re.DOTALL|re.UNICODE)

In [27]:
match_format.findall(
    "Let me think!</think>"\
    f"Hence, the solution is 2.",
)

['Hence, the solution is 2.']

In [28]:
match_format.findall(
    "<think>Let me think!</think>"\
    f"\n\nHence, the solution is 2",
)

['\n\nHence, the solution is 2']

In [57]:
# Inspect dataset for empty or invalid entries
empty_prompts = [i for i, example in enumerate(combined_dataset) if not example["prompt"] or not example["prompt"].strip() or not example["chosen"] or "<answer></answer>" in example["chosen"] or not example["rejected"]]
if empty_prompts:
    print(f"Found {len(empty_prompts)} empty or invalid prompts at indices: {empty_prompts}")
    for idx in empty_prompts[:5]:
        print(f"Index {idx}: {combined_dataset[idx]}")
else:
    print("No empty or invalid prompts found in combined_dataset.")

No empty or invalid prompts found in combined_dataset.


In [58]:
# Log first few examples for debugging
print("First 3 examples in combined_dataset:")
for i in range(min(3, len(combined_dataset))):
    print(f"Example {i}: {combined_dataset[i]}")

First 3 examples in combined_dataset:
Example 0: {'prompt': 'What is the best way for a marine scientist to determine the eating habits of sharks?', 'chosen': '<reasoning>Reasoning: Analyze the question and deduce the correct solution.</reasoning><answer>3</answer>', 'rejected': '<reasoning>Incomplete or incorrect reasoning.</reasoning><answer>Wrong or irrelevant response.</answer>'}
Example 1: {'prompt': "Millions of sports lovers are still thinking about one thing----the Winter Olympics in Sochi Russia. Close to 2900 men and women competed in the 2014 Winter Games about a month ago. The Olympics brought a lot of attention to Sochi, a city on the Black Sea, which is a popular area for vacation travelers. The area is known for its mild winters. At least $50 billion was spent on the 2014 Winter Games, making it the costliest Olympics in history. Seven billion dollars was spent on the 2010 Winter Games in Vancouver, Canada. Olympic officials chose Sochi to host the 2014 games almost seve

In [59]:
# Verify final schema
print(f"Combined dataset features: {combined_dataset.features}, Num rows: {combined_dataset.num_rows}")

Combined dataset features: {'prompt': Value(dtype='string', id=None), 'chosen': Value(dtype='string', id=None), 'rejected': Value(dtype='string', id=None)}, Num rows: 140275


In [60]:
# Regex for answer extraction
solution_end_regex = rf"{reasoning_end}(.*)"
match_format = re.compile(solution_end_regex, re.DOTALL)
match_numbers = re.compile(r".*?[\s]{0,}([-]?[\d\.\,]{1,})", flags=re.MULTILINE | re.DOTALL)

In [61]:
## The reward function creation that matches the format exactly
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0] if isinstance(completion, list) else str(completion)
        if match_format.search(response):
            score += 3.0
        scores.append(score)
    return scores

In [62]:
##If above one fails: we reward the model if it atleast follows the format partially, by counting each symbol
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0] if isinstance(completion, list) else str(completion)
        score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end) == 1 else -1.0
        scores.append(score)
    return scores

In [63]:
## To extract the generated answers and reward or penalize it!
def check_answer(prompts, completions, **kwargs):
    question = prompts[0][-1]
    responses = [completion[0] for completion in completions]
    chosen = kwargs.get('chosen', [None] * len(completions))
    true_answers = [
        re.search(r'<answer>(.*?)</answer>', c).group(1) if c and re.search(r'<answer>(.*?)</answer>', c) else ''
        for c in chosen
    ]
    extracted_responses = [
        guess.group(1) if (guess := match_format.search(r)) else None
        for r in responses
    ]
    scores = []
    for guess, true_answer in zip(extracted_responses, true_answers):
        score = 0
        if guess is None or not true_answer:
            scores.append(-2.0)
            continue
        if guess == true_answer:
            score += 5.0
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            try:
                ratio = float(guess) / float(true_answer)
                if 0.9 <= ratio <= 1.1:
                    score += 2.0
                elif 0.8 <= ratio <= 1.2:
                    score += 1.5
                else:
                    score -= 2.5
            except:
                score -= 4.5
        scores.append(score)
    return scores

In [32]:
match_numbers = re.compile(
    r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("  0.34  "))
print(match_numbers.findall("  123,456  "))
print(match_numbers.findall("  -0.234  "))
print(match_numbers.findall("17"))

['0.34']
['123,456']
['-0.234']
['17']


In [33]:
import langid

def get_lang(text: str) -> str:
    if not text:
        return "und"
    lang, _ = langid.classify(text)
    return lang


print(get_lang("Hello, How are you")) # This should return en
print(get_lang("Aku berpikir kalau aku adalah kamu")) # This should return id
print(get_lang("我在这里")) # This should return zh
print(get_lang("नमस्ते! कस्तो छ हजुर?")) # This should return ne

en
id
zh
ne


In [64]:
import re
import langid

def format_and_language_reward_func(completions, **kwargs):
    scores = []
    for completion in completions:
        response_content = completion[0] if isinstance(completion, list) else str(completion)
        lang = langid.classify(response_content)[0] if response_content else "und"
        if lang == 'ne':
            score = 5.0
        elif lang == 'en':
            score = 4.0
        elif lang == 'zh':
            score = -7.0
        else:
            score = -7.0
        scores.append(score)
    return scores

In [65]:
# prompts = [
#     [{"role": "assistant", "content": "What is the result of (1 + 2) * 4?"}],
#     [{"role": "assistant", "content": "What is the result of (3 + 1) * 2?"}],
# ]
# completions = [
#     [{"role": "assistant", "content": "<think>The sum of 1 and 2 is 3, which we multiply by 4 to get 12.</think><answer>(1 + 2) * 4 = 12</answer>"}],
#     [{"role": "assistant", "content": "The sum of 3 and 1 is 4, which we multiply by 2 to get 8. So (3 + 1) * 2 = 8."}],
# ]
# format_and_language_reward_func(prompts=prompts, completions=completions)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [66]:
## Prepping our main function that prints out the generated responses and the true answer, along with another reward function which converts text to float via float and sees if it's the same.
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, **kwargs):
    question = prompts[0][-1]
    responses = [completion[0] for completion in completions]
    chosen = kwargs.get('chosen', [None] * len(completions))
    true_answers = [
        re.search(r'<answer>(.*?)</answer>', c).group(1) if c and re.search(r'<answer>(.*?)</answer>', c) else ''
        for c in chosen
    ]
    extracted_responses = [
        guess.group(1) if (guess := match_numbers.search(r)) else None
        for r in responses
    ]
    scores = []
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{true_answers[0]}",
            f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1
    for guess, true_answer in zip(extracted_responses, true_answers):
        if guess is None or not true_answer:
            scores.append(-2.5)
            continue
        try:
            true_answer = float(true_answer.strip())
            guess = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
        continue
    return scores

In [67]:
# Tokenize prompts with system prompt
def tokenize_prompt(example):
    chat_format = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": example["prompt"]}
    ]
    tokens = tokenizer.apply_chat_template(chat_format, add_generation_prompt=True, tokenize=True)
    return {"tokens": tokens}

In [68]:
# Apply tokenization and filter by length
tokenized = combined_dataset.map(tokenize_prompt)
tokenized = tokenized.map(lambda x: {"L": len(x["tokens"])})
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)
dataset = combined_dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Map:   0%|          | 0/140275 [00:00<?, ? examples/s]

Map:   0%|          | 0/140275 [00:00<?, ? examples/s]

Max Length =  514


### Training the Model

In [69]:
## Now we set up the GRPO Trainer and all configs
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [70]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
        format_and_language_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 126,381 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 87,293,952/8,000,000,000 (1.09% trained)


********************Question:
. 
Answer:
1 
Response:
  
Extracted:
None


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers,rewards / format_and_language_reward_func
1,0.000000,-6.625000,4.337713,507.750000,0.000020,0.000000,-2.000000,-2.000000,-2.500000,-0.125000
2,0.000000,-4.562500,2.962713,509.000000,0.000024,0.000000,-2.000000,-2.000000,-2.500000,1.937500
3,0.000000,-7.312500,2.962713,509.000000,0.000020,0.000000,-2.000000,-2.000000,-2.500000,-0.812500
4,0.000000,-5.250000,3.175426,509.000000,0.000022,0.000000,-2.000000,-2.000000,-2.500000,1.250000


********************Question:
s 
Answer:
0 
Response:
  
Extracted:
None
********************Question:
. 
Answer:
3 
Response:
" 
Extracted:
None
********************Question:
. 
Answer:
1 
Response:
  
Extracted:
None
********************Question:
. 
Answer:
21 
Response:
  
Extracted:
None


KeyboardInterrupt: 